In [1]:
from collections import defaultdict

In [110]:
import random
class Trigram:
    length = 0

    def __init__(self, fn=None):
        self.lut = {}
        if fn is not None:
            self.parseFile(fn)

    def parseFile(self, fn):
        pair = '  '
        for z, line in enumerate(fn):
            # \n's are spurious in a prose context
            letter = str(line)
            d = self.lut.setdefault(pair, {})
            d[letter] = d.get(letter, 0) + 1
            pair = pair[1] + letter
        self.measure()


    def measure(self):
        """calculates the scalar length of the trigram vector and
        stores it in self.length."""
        total = 0
        for y in self.lut.values():
            total += sum([ x * x for x in y.values() ])
        self.length = total ** 0.5

    def similarity(self, other):
        """returns a number between 0 and 1 indicating similarity.
        1 means an identical ratio of trigrams;
        0 means no trigrams in common.
        """
        if not isinstance(other, Trigram):
            raise TypeError("can't compare Trigram with non-Trigram")
        lut1 = self.lut
        lut2 = other.lut
        total = 0
        for k in lut1.keys():
            if k in lut2:
                a = lut1[k]
                b = lut2[k]
                for x in a:
                    if x in b:
                        total += a[x] * b[x]

        return float(total) / (self.length * other.length)

    def __sub__(self, other):
        """indicates difference between trigram sets; 1 is entirely
        different, 0 is entirely the same."""
        return 1 - self.similarity(other)


    def makeWords(self, count):
        """returns a string of made-up words based on the known text."""
        text = []
        k = '  '
        while count:
            n = self.likely(k)
            text.append(n)
            k = k[1] + n
            if n in ' \t':
                count -= 1
        return ''.join(text)


    def likely(self, k):
        """Returns a character likely to follow the given string
        two character string, or a space if nothing is found."""
        if k not in self.lut:
            return ' '
        # if you were using this a lot, caching would a good idea.
        letters = []
        for k, v in self.lut[k].items():
            letters.append(k * v)
        letters = ''.join(letters)
        return random.choice(letters)

In [84]:
def decrypt(F, key, KEY_LENGTH, xor=True):
    crypt_text = []
    for i in range(len(F)):
        b = key[i % KEY_LENGTH]
        if xor:
            F[i] ^= b
        else:
            F[i] = (F[i] - b) % 256
    return F

In [27]:
def compute_distribution(v):
    d= defaultdict(int)
    for e in v: d[e]+=1
    s= float(sum(d.values()))
    return dict((k, v/s) for k, v in d.items())

def compute_distribution_2(v):
    d= defaultdict(int)
    for e in v: d[e]+=1
    return d

In [3]:
def extract_n(string, n):
    keep = []
    for i, char in enumerate(string):
        if i % n == 0:
            keep.append(char)
        else:
            continue
    return keep

In [13]:
def possible_key_length(bytes,top_n):
    keep = []
    for key_l in range(1,top_n):
        q = extract_n(bytes, key_l)
        C = compute_distribution(q)
        S = score(C)
        E = entropy(q)
        IC = compute_IC(q)
        keep.append((key_l, IC, S,E))
    return keep

In [119]:
def score(distribution):
    s = 0
    for q in distribution.values():
        s += q ** 2
    return s

In [6]:
from math import log
log2= lambda x:log(x,2)
def entropy(y):
    # P(Y)
    Py= compute_distribution(y)
    res=0.0
    for k, v in Py.iteritems():
        res+=v*log2(v)
    return -res

In [7]:
import sys
import collections


def compute_IC(cipher_flat):
    N            = len(cipher_flat)
    freqs        = collections.Counter( cipher_flat )
    alphabet     = range(0,256)
    freqsum      = 0.0

    for letter in alphabet:
        freqsum += freqs[ letter ] * ( freqs[ letter ] - 1 )

    IC = freqsum / ( N*(N-1) )
    return IC

In [14]:
t = bytearray(open('/home/vagrant/ctf/vigtor.bin', 'rb').read())

In [100]:
def get_key_len(t):
    key_len_poss = possible_key_length(t,60)
    candidates = sorted(key_len_poss, key=lambda x: x[1], reverse=True)[0:20]
    ordered_candidates = sorted(candidates, key=lambda x: x[0])
    return ordered_candidates


In [16]:
ordered_candidates

[(4, 0.009322839905198886, 0.009329937412453422, 7.350275349827261),
 (5, 0.01677023429449359, 0.01677903946859206, 6.922413944001221),
 (10, 0.030688861619027865, 0.030706222519183344, 6.389577489995471),
 (12, 0.009382275313276985, 0.009403566557601497, 7.342954785669705),
 (15, 0.016699864999284247, 0.016726282175550996, 6.926905746726175),
 (16, 0.009466245070444189, 0.009494630379789941, 7.319002930943954),
 (20, 0.03179321865832377, 0.031827900278173366, 6.322151000815202),
 (25, 0.016713331008918392, 0.01675735942735707, 6.920209960597651),
 (28, 0.009262545170238161, 0.009312228609124363, 7.34672447301796),
 (30, 0.030779714584676324, 0.030831792403461726, 6.384810562947812),
 (32, 0.00942496127723638, 0.009481734262032504, 7.319225569806714),
 (35, 0.016294259824543522, 0.01635592256761225, 6.936651311988929),
 (36, 0.009362365629828158, 0.00942624064655201, 7.335857229674096),
 (40, 0.030652394381880714, 0.03072183686383631, 6.32486807307724),
 (45, 0.016839496037495107, 0.01

In [88]:
import base64

a = '31 ED 49 89 D1 5E 48 89 E2 48 83 E4 F0 50 54 49C7 C0 B0 0A 40 00 48 C7 C1 40 0A 40 00 48 C7 C7'
a = a.replace(' ', '')
a = base64.b16decode(a)

b = 'DA 67 A9 83 43 ED A7 80 14 EC D1 A7 1E A8 87 EDA7 C1 28 26 1B 98 9A F5 DC 76 7C D3 1C E9 C7 98'
b = b.replace(' ', '')
b = base64.b16decode(b)

def encrypt_2(F, key, KEY_LENGTH, xor=True):
   #F = bytearray(open(filename, 'rb').read())
   a = F
   b = key
   a = map(lambda x: ord(x), list(a))
   b = map(lambda x: ord(x), list(b))
   F = a
   key = b
   crypt_text = []
   for i in range(len(F)):
       b = key[i % KEY_LENGTH]
       if xor:
           F[i] ^= b
       else:
           F[i] = (F[i] - b) % 256
   return ''.join(map(lambda x: chr(x), F))

def xorText (a, b) :
    a = map(lambda x: ord(x), list(a))
    b = map(lambda x: ord(x), list(b))

    r = []
    for i in range(len(a)) :
        r.append(a[i] ^ b[i % len(b)])

    return ''.join(map(lambda x: chr(x), r))

print base64.b16encode(xorText(a, b))
print base64.b16encode(encrypt_2(a, b, len(b), xor=False))

print ' '.join(map(lambda x: '%02x' % (ord(x)), xorText(a, b)))
print ' '.join(map(lambda x: '%02x' % (ord(x)), encrypt_2(a, b, len(b), xor=False)))

print ' '.join(map(lambda x: '%02x' % (ord(x)), xorText(b, '\xed\xa7\x80\xf5h')))

EB8AE00A92B3EF09F6A45243EEF8D3A46001982C5B98D2321D3676931CA1005F
5786A0068E71A109CE5CB23DD2A8CD5C20FF88E42568AED2E5CA8E6DE45F002F
eb 8a e0 0a 92 b3 ef 09 f6 a4 52 43 ee f8 d3 a4 60 01 98 2c 5b 98 d2 32 1d 36 76 93 1c a1 00 5f
57 86 a0 06 8e 71 a1 09 ce 5c b2 3d d2 a8 cd 5c 20 ff 88 e4 25 68 ae d2 e5 ca 8e 6d e4 5f 00 2f
37 c0 29 76 2b 00 00 00 e1 84 3c 00 9e 5d ef 00 00 41 dd 4e f6 3f 1a 00 b4 9b db 53 e9 81 2a 3f


In [89]:
def encrypt(F, key, KEY_LENGTH, xor=True):
    crypt_text = []
    for i in range(len(F)):
        b = key[i % KEY_LENGTH]
        if xor:
            F[i] ^= b
        else:
            F[i] = (F[i] + b) % 256
    return F

In [81]:
def extract_n(string, n):
    keep = []
    for i, char in enumerate(string):
        if i % n == 0:
            keep.append(char)
        else:
            continue
    return keep

def get_cipher_stream(data,i,k_l):
    cipher_stream = []
    k = 0
    O_ = i+k_l*k
    while(O_ < len(data)):
        #print O_
        cipher_stream.append(data[O_])
        k += 1
        O_ = i+ k_l*k

    return cipher_stream

In [191]:
def get_chi_square(s):
       
        from scipy.stats import chisquare
        tar_b = bytearray(open('/home/vagrant/ctf/tar.bin', 'rb').read())
        B = compute_distribution_2(tar_b)
        ordered_values_b = map(lambda x: x[1], list(sorted( B.items() , key = lambda x: x[0])))
        e_frq = ordered_values_b
        
        A  = compute_distribution_2(s)
        ordered_values_a = map(lambda x: x[1], list(sorted( A.items() , key = lambda x: x[0])))
        chi_sqr = chisquare(ordered_values_a, f_exp=ordered_values_b)

        return chi_sqr[0]

In [192]:
tar = bytearray(open('/home/vagrant/ctf/tar.bin', 'rb').read())

tar_t = Trigram(tar)

def diff_dist(data):
    d = Trigram(data)
    return tar_t - d

In [113]:
def find_ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

In [403]:
import numpy as np
from scipy.linalg import norm


_SQRT2 = np.sqrt(2)     # sqrt(2) with default precision np.float64


def hellinger1(p, q):
    return norm(np.sqrt(p) - np.sqrt(q)) / _SQRT2

b = bytearray(open('/home/vagrant/ctf/tar.bin', 'rb').read())

B = compute_distribution(find_ngrams(b,1))

ordered_values_B = map(lambda x: x[1], list(sorted( B.items() , key = lambda x: x[0])))

def hellinger(D_P):
    A = compute_distribution(find_ngrams(D_P,1))
    
    ordered_values_a = map(lambda x: x[1], list(sorted( A.items() , key = lambda x: x[0])))
    return hellinger1(ordered_values_a, ordered_values_B)

In [61]:
vigtor_b = bytearray(open('/home/vagrant/ctf/vigtor.bin', 'rb').read())

In [78]:
decValues = range(256)

In [82]:
C1 = get_cipher_stream(t,0,5)

In [85]:
D_P = decrypt(C1, [0xff], 1, xor=True)

In [86]:
get_chi_square(D_P)

858650.52341645444

In [460]:
key = [0xA1, 0x2F, 0xFF, 0xA2, 0xB6] #, 0x2F, 0xFF, 0xA2]
KEY_LENGTH = len(key)
ls_b = bytearray(open('/home/vagrant/ctf/ls.bin', 'rb').read())
print ls_b[0]
g = encrypt(ls_b, key, KEY_LENGTH, xor=True)
open('ctext.out', 'wb').write(g)
t = bytearray(open('ctext.out', 'rb').read())

80


In [105]:
t[0]

241

In [92]:
ls_b[0]

241

In [108]:
ls_b = bytearray(open('/home/vagrant/ctf/ls.bin', 'rb').read())
ls_b[0] #^ 0xA1

80

In [101]:
get_key_len(t)

[(2, 0.03471668925967946, 0.03474730110475418, 6.3235062933963295),
 (4, 0.035594360893965026, 0.03565552697750065, 6.221614406156556),
 (6, 0.03555260534906071, 0.03564436135654343, 6.297132154294561),
 (10, 0.034912250450330036, 0.035065268961436684, 6.316942983819626),
 (12, 0.03594264793839889, 0.03612606828696462, 6.19856645990381),
 (14, 0.03399439738360574, 0.03420882704012436, 6.297557168920161),
 (18, 0.037363603538780196, 0.037638328537770306, 6.221703639604274),
 (20, 0.034831699978339924, 0.0351377140240031, 6.208903334994246),
 (22, 0.0336334412501573, 0.0339705066700212, 6.277204575054588),
 (26, 0.03405750418575399, 0.0344556667973839, 6.239240970935525),
 (28, 0.03409308606878218, 0.03452180640341654, 6.192871575153851),
 (30, 0.03525026320516249, 0.03570901248561651, 6.233637862984173),
 (34, 0.035859815013506165, 0.03637956713479271, 6.198865741412255),
 (36, 0.04025683953592077, 0.04080463814349155, 6.0425181621674255),
 (42, 0.03698998714571999, 0.03763113895188127,

In [466]:
filename = './ctf/vigtor.bin'
def stage1(bootstrap_1):
    H = []
    for i in bootstrap_1:#decValues:
        print i
        for k in decValues:#[47, 161, 208, 94, 233, 103, 166, 40, 164, 32]:
            t = bytearray(open(filename, 'rb').read())
            P = [i,k]
            D_P = decrypt(t, P, 2, xor=True)

            chi = get_chi_square(D_P)
            diff = chi #diff_dist(D_P)
            H_S = hellinger(D_P)
            H.append((i,k,chi, diff, H_S))
    return H

In [467]:
def stage2(bootstrap_1, bootstrap_2):
    H = []
    for i in bootstrap_1:#decValues:
        print i
        for k in bootstrap_2:#[47, 161, 208, 94, 233, 103, 166, 40, 164, 32]:
            for j in decValues:
                t = bytearray(open(filename, 'rb').read())
                P = [i,k,j]
                D_P = decrypt(t, P, 3, xor=True)

                chi = get_chi_square(D_P)
                diff = chi #diff_dist(D_P)
                H_S = hellinger(D_P)
                H.append((i,k,j,chi, diff, H_S))
    return H

In [468]:
def stage3(bootstrap_1, bootstrap_2, bootstrap_3):
    H = []
    for i in bootstrap_1:#decValues:
        print i
        for k in bootstrap_2:#[47, 161, 208, 94, 233, 103, 166, 40, 164, 32]:
            for j in bootstrap_3:
                for h in decValues:
                    t = bytearray(open(filename, 'rb').read())
                    P = [i,k,j,h]
                    D_P = decrypt(t, P, 4, xor=True)

                    chi = get_chi_square(D_P)
                    diff = chi#diff_dist(D_P)
                    H_S = hellinger(D_P)
                    H.append((i,k,j,h ,chi, diff, H_S))
    return H

In [469]:
def stage4(bootstrap_1, bootstrap_2, bootstrap_3, bootstrap_4):
    print bootstrap_1, bootstrap_2, bootstrap_3, bootstrap_4
    H = []
    for i in bootstrap_1:#decValues:
        print i
        for k in bootstrap_2:#[47, 161, 208, 94, 233, 103, 166, 40, 164, 32]:
            for j in bootstrap_3:
                for h in bootstrap_4:
                    for g in decValues:
                        t = bytearray(open(filename, 'rb').read())
                        P = [i,k,j,h,g]
                        D_P = decrypt(t, P, 5, xor=True)

                        chi = get_chi_square(D_P)
                        diff = diff_dist(D_P)
                        H_S = hellinger(D_P)
                        if P == [161, 47, 255, 162, 182]:
                            print chi, diff, H_S
                            break
                        H.append((i,k,j,h,g, chi, diff, H_S))
    return H

In [470]:
def stage_1_g(H):
    chi_1 = sorted(H, key=lambda x: x[-3])[0:5]
    P_chi_1 = map(lambda x: x[0:2], chi_1)
    a1, b1 = map(lambda x: set(x), zip(*P_chi_1))

    diff_1 = sorted(H, key=lambda x: x[-2])[0:5]
    P_diff_1 = map(lambda x: x[0:2], diff_1)
    a2, b2 = map(lambda x: set(x), zip(*P_diff_1))


    dist_h = sorted(H, key=lambda x: x[-1])[0:5]
    P_diff_h = map(lambda x: x[0:2], dist_h)
    a3, b3 = map(lambda x: set(x), zip(*P_diff_h))

    boostrap1 = a1.union(a2,a3)
    bootstrap2 = b1.union(b2,b3)
    return boostrap1, bootstrap2


In [471]:
def stage_2_g(H):
    chi_1 = sorted(H, key=lambda x: x[-3])[0:5]
    P_chi_1 = map(lambda x: x[0:3], chi_1)
    a1, b1, c1 = map(lambda x: set(x), zip(*P_chi_1))

    diff_1 = sorted(H, key=lambda x: x[-2])[0:5]
    P_diff_1 = map(lambda x: x[0:3], diff_1)
    a2, b2, c2 = map(lambda x: set(x), zip(*P_diff_1))


    dist_h = sorted(H, key=lambda x: x[-1])[0:5]
    P_diff_h = map(lambda x: x[0:3], dist_h)
    a3, b3, c3 = map(lambda x: set(x), zip(*P_diff_h))

    boostrap1 = a1.union(a2,a3)
    bootstrap2 = b1.union(b2,b3)
    bootstrap3 = c1.union(c2,c3)
    return boostrap1, bootstrap2, bootstrap3

In [472]:
def stage_3_g(H):
    chi_1 = sorted(H, key=lambda x: x[-3])[0:5]
    P_chi_1 = map(lambda x: x[0:4], chi_1)
    a1, b1, c1, d1 = map(lambda x: set(x), zip(*P_chi_1))

    diff_1 = sorted(H, key=lambda x: x[-2])[0:5]
    P_diff_1 = map(lambda x: x[0:4], diff_1)
    a2, b2, c2, d2 = map(lambda x: set(x), zip(*P_diff_1))


    dist_h = sorted(H, key=lambda x: x[-1])[0:5]
    P_diff_h = map(lambda x: x[0:4], dist_h)
    a3, b3, c3, d3 = map(lambda x: set(x), zip(*P_diff_h))

    boostrap1 = a1.union(a2,a3)
    bootstrap2 = b1.union(b2,b3)
    bootstrap3 = c1.union(c2,c3)
    bootstrap4 = d1.union(d2,d3)
    return boostrap1, bootstrap2, bootstrap3, bootstrap4

In [473]:
def stage_4_g(H):
    chi_1 = sorted(H, key=lambda x: x[-3])[0:5]
    P_chi_1 = map(lambda x: x[0:5], chi_1)
    a1, b1, c1, d1, e1 = map(lambda x: set(x), zip(*P_chi_1))

    diff_1 = sorted(H, key=lambda x: x[-2])[0:5]
    P_diff_1 = map(lambda x: x[0:5], diff_1)
    a2, b2, c2, d2, e2 = map(lambda x: set(x), zip(*P_diff_1))


    dist_h = sorted(H, key=lambda x: x[-1])[0:5]
    P_diff_h = map(lambda x: x[0:5], dist_h)
    a3, b3, c3, d3, e3 = map(lambda x: set(x), zip(*P_diff_h))

    boostrap1 = a1.union(a2,a3)
    bootstrap2 = b1.union(b2,b3)
    bootstrap3 = c1.union(c2,c3)
    bootstrap4 = d1.union(d2,d3)
    bootstrap5 = e1.union(e2,e3)
    return boostrap1, bootstrap2, bootstrap3, bootstrap4, bootstrap5

In [476]:
b1 = [237, 167, 128, 104, 245, 152, 219, 135, 212, 154] #[182, 255, 162, 47, 161, 73, 254, 94, 0, 208]

H1 = stage1(b1)
b1, b2 = stage_1_g(H)

print b1, b2

print "Done with stage 1"
H2 = stage2(b1,b2)
b1, b2, b3 = stage_2_g(H2)

print b1, b2, b3

237
167
128
104
245
152
219
135
212
154
set([161, 103, 94, 47]) set([161, 94, 166, 47])
Done with stage 1
161
103
94
47
set([161, 103, 47]) set([161, 166, 47]) set([152, 128, 227])


In [ ]:
print "Done with stage 1"
H2 = stage2(b1,b2)
b1, b2, b3 = stage_2_g(H2)

print b1, b2, b3

Done with stage 1
161
103
47
set([161, 103, 47]) set([161, 166, 47]) set([152, 128, 227])


In [ ]:
print "Done with stage 2"
H3 = stage3(b1,b2,b3)
b1, b2, b3, b4 = stage_3_g(H3)

print b1,b2,b3,b4

Done with stage 2
161


In [ ]:
print "Done with stage 3"
H4 = stage4(b1, b2, b3, b4)

In [312]:
stage_4_g(H4)

H = H4

In [313]:
H4

[(161,
  166,
  162,
  182,
  0,
  195518.69604766706,
  0.993037095045122,
  0.41698101594848597),
 (161,
  166,
  162,
  182,
  1,
  211088.03112036086,
  0.9878746427327287,
  0.40953828138227621),
 (161,
  166,
  162,
  182,
  2,
  225320.06490429936,
  0.9929776128486322,
  0.42907996125878733),
 (161,
  166,
  162,
  182,
  3,
  222379.08589576831,
  0.9935836430723651,
  0.43150491412904246),
 (161,
  166,
  162,
  182,
  4,
  232854.11340461546,
  0.993707612800744,
  0.426461751246763),
 (161,
  166,
  162,
  182,
  5,
  218364.96056824096,
  0.9925675076096632,
  0.42287439059972265),
 (161,
  166,
  162,
  182,
  6,
  214183.60980950261,
  0.9922131597162824,
  0.43018459730314512),
 (161,
  166,
  162,
  182,
  7,
  234884.20685620696,
  0.9933057311119565,
  0.43337890458684325),
 (161,
  166,
  162,
  182,
  8,
  194966.05699280679,
  0.9930612783890476,
  0.41388117303554839),
 (161,
  166,
  162,
  182,
  9,
  196292.1838462876,
  0.99260290613061,
  0.41927257322002787

In [314]:
chi_1 = sorted(H, key=lambda x: x[-3])[0:10]
P_chi_1 = map(lambda x: x[0:5], chi_1)
a1, b1, c1, d1, e1 = map(lambda x: set(x), zip(*P_chi_1))

diff_1 = sorted(H, key=lambda x: x[-2])[0:10]
P_diff_1 = map(lambda x: x[0:5], diff_1)
a2, b2, c2, d2, e2 = map(lambda x: set(x), zip(*P_diff_1))


dist_h = sorted(H, key=lambda x: x[-1])[0:10]
P_diff_h = map(lambda x: x[0:5], dist_h)
a3, b3, c3, d3, e3 = map(lambda x: set(x), zip(*P_diff_h))

boostrap1 = a1.union(a2,a3)
bootstrap2 = b1.union(b2,b3)
bootstrap3 = c1.union(c2,c3)
bootstrap4 = d1.union(d2,d3)
bootstrap5 = e1.union(e2,e3)
#return boostrap1, bootstrap2, bootstrap3, bootstrap4, bootstrap5

In [317]:
dist_h, [161, 47, 255, 162, 182]  (122152.56827095788, 0.02876273534719398, 0.096460923131097998)

([(161,
   47,
   182,
   182,
   182,
   147316.06845966983,
   0.4138115570538502,
   0.24840646732694499),
  (161,
   47,
   182,
   194,
   182,
   150035.81726915605,
   0.4944501821345746,
   0.25236912525843314),
  (161,
   47,
   162,
   194,
   182,
   168290.18249521204,
   0.5037499923420931,
   0.28358694018216435),
  (161,
   47,
   162,
   182,
   182,
   170072.61618096841,
   0.42081441256928975,
   0.28530491724825369),
  (161,
   47,
   182,
   182,
   73,
   158227.57129070201,
   0.8224821735257661,
   0.2985927745252186),
  (161,
   47,
   182,
   194,
   73,
   160008.9082313168,
   0.8324090631784699,
   0.2997141670268797),
  (161,
   47,
   182,
   182,
   61,
   154054.76043205458,
   0.7494623129136952,
   0.30005523992754524),
  (161,
   47,
   182,
   194,
   61,
   157106.68486751342,
   0.7468192787713592,
   0.30221200206539783),
  (161,
   47,
   182,
   182,
   183,
   155021.89637478549,
   0.6105017079307853,
   0.3031835838101371),
  (161,
   47,
  

In [318]:
ctext_b = bytearray(open('/home/vagrant/ctext.out', 'rb').read())
D_P = decrypt(t, [161, 47, 255, 162, 182], 5, xor=True)
chi = get_chi_square(D_P)
diff = diff_dist(D_P)
H_S = hellinger(D_P)

chi, diff, H_S

In [329]:
chi = get_chi_square(D_P)
diff = diff_dist(D_P)
H_S = hellinger(D_P)

chi, diff, H_S

(122152.56827095788, 0.02876273534719398, 0.096460923131097998)

In [325]:
tar_b[100]

0

In [136]:
256 / 2


128

In [320]:
        tar_b = bytearray(open('/home/vagrant/ctf/ls.bin', 'rb').read())
        B = compute_distribution_2(tar_b)
        ordered_values_b = map(lambda x: (x[0], x[1]), list(sorted( B.items() , key = lambda x: x[1])))
        e_frq = ordered_values_b

In [157]:
ordered_values_b

[(145, 16),
 (153, 16),
 (169, 16),
 (161, 17),
 (174, 18),
 (155, 19),
 (159, 19),
 (158, 20),
 (206, 20),
 (143, 21),
 (171, 21),
 (178, 21),
 (50, 22),
 (122, 23),
 (157, 23),
 (166, 23),
 (172, 23),
 (162, 24),
 (113, 25),
 (167, 25),
 (82, 26),
 (180, 26),
 (58, 27),
 (81, 27),
 (103, 27),
 (173, 27),
 (138, 28),
 (150, 28),
 (164, 28),
 (140, 29),
 (163, 29),
 (212, 29),
 (154, 30),
 (220, 30),
 (90, 31),
 (147, 31),
 (101, 32),
 (183, 32),
 (188, 32),
 (34, 33),
 (106, 33),
 (177, 33),
 (203, 33),
 (26, 34),
 (89, 34),
 (121, 34),
 (207, 34),
 (156, 35),
 (146, 36),
 (151, 36),
 (168, 37),
 (35, 38),
 (130, 38),
 (179, 38),
 (18, 39),
 (62, 39),
 (181, 39),
 (211, 39),
 (111, 40),
 (28, 41),
 (107, 42),
 (110, 42),
 (98, 43),
 (165, 43),
 (39, 44),
 (17, 46),
 (22, 46),
 (109, 46),
 (142, 46),
 (14, 48),
 (23, 48),
 (27, 48),
 (30, 48),
 (202, 49),
 (205, 49),
 (229, 49),
 (42, 50),
 (43, 50),
 (221, 50),
 (38, 51),
 (74, 52),
 (78, 52),
 (213, 52),
 (215, 52),
 (54, 53),
 (105,

In [474]:
tar_b = bytearray(open('/home/vagrant/ctf/vigtor.bin', 'rb').read())
B = compute_distribution_2(tar_b)
ordered_values_b = map(lambda x: (x[0], x[1]), list(sorted( B.items() , key = lambda x: x[1], reverse=True)))
e_frq = ordered_values_b

In [475]:
map(lambda x: x[0], ordered_values_b[0:10])

[237, 167, 128, 104, 245, 152, 219, 135, 212, 154]

In [160]:
0x47

71

In [164]:
f = [(135, 7484),
 (219, 7761),
 (152, 7857),
 (245, 8763),
 (104, 9101),
 (128, 9109),
 (167, 10346),
 (237, 10428)]

In [165]:
for i in f:
    print i[0]

135
219
152
245
104
128
167
237


In [ ]:
using a key formed by the most frequently occuring byte in each residue class moduluo the key length.

In [461]:
v_b = bytearray(open('/home/vagrant/ctext.out', 'rb').read())


In [462]:
def compute_distribution_3(v,r):
    d= defaultdict(list)
    for e in v: d[e % r].append(e)
    return d

In [463]:
g = []
for a in range(1,50):
    W = compute_distribution_3(v_b,a)
    key = [] 
    for i in range(0,a):
        m = max(compute_distribution_2(W[i]).items(),key=lambda item:item[1])
        print m, i
        key.append(m[0])
    g.append(key)

(182, 1956) 0
(182, 1956) 0
(255, 1956) 1
(255, 1956) 0
(73, 1198) 1
(182, 1956) 2
(0, 1072) 0
(161, 1900) 1
(182, 1956) 2
(255, 1956) 3
(255, 1956) 0
(161, 1900) 1
(182, 1956) 2
(73, 1198) 3
(254, 1099) 4
(162, 1934) 0
(73, 1198) 1
(182, 1956) 2
(255, 1956) 3
(94, 1096) 4
(47, 1909) 5
(182, 1956) 0
(162, 1934) 1
(254, 1099) 2
(255, 1956) 3
(32, 480) 4
(47, 1909) 5
(118, 623) 6
(0, 1072) 0
(161, 1900) 1
(162, 1934) 2
(43, 527) 3
(164, 487) 4
(93, 1024) 5
(182, 1956) 6
(255, 1956) 7
(162, 1934) 0
(73, 1198) 1
(182, 1956) 2
(255, 1956) 3
(94, 1096) 4
(185, 514) 5
(42, 494) 6
(43, 527) 7
(161, 1900) 8
(0, 1072) 0
(161, 1900) 1
(182, 1956) 2
(73, 1198) 3
(254, 1099) 4
(255, 1956) 5
(166, 547) 6
(47, 1909) 7
(208, 1030) 8
(199, 359) 9
(0, 1072) 0
(254, 1099) 1
(255, 1956) 2
(47, 1909) 3
(103, 798) 4
(93, 1024) 5
(182, 1956) 6
(161, 1900) 7
(162, 1934) 8
(185, 514) 9
(208, 1030) 10
(0, 1072) 0
(73, 1198) 1
(182, 1956) 2
(255, 1956) 3
(208, 1030) 4
(161, 1900) 5
(162, 1934) 6
(103, 798) 7
(16

In [465]:
X_ = True

if X_:
    x = "T"
else:
    x = "F"
for i, key in enumerate(g):
    ctext_b = bytearray(open('/home/vagrant/ctext.out', 'rb').read())
    D_P = decrypt(ctext_b, key, len(key), xor=X_)
    out = '/home/vagrant/ctf/ctext_%s_%d.guess' % (x,i)
    #open(out, 'wb').write(ctext_b)
    chi = get_chi_square(D_P)
    diff = diff_dist(D_P)
    H_S = hellinger(D_P)
    #make_bin(out)
    print i, chi, diff, H_S, out, key, [0xA1, 0x2F, 0xFF, 0xA2, 0xB6]

0 232274.332428 0.990943644467 0.418423759462 /home/vagrant/ctf/ctext_T_0.guess [182] [161, 47, 255, 162, 182]
1 190332.109301 0.995829415032 0.415203024999 /home/vagrant/ctf/ctext_T_1.guess [182, 255] [161, 47, 255, 162, 182]
2 184603.232997 0.991368298272 0.422661515005 /home/vagrant/ctf/ctext_T_2.guess [255, 73, 182] [161, 47, 255, 162, 182]
3 190079.416507 0.989386600836 0.420024850623 /home/vagrant/ctf/ctext_T_3.guess [0, 161, 182, 255] [161, 47, 255, 162, 182]
4 246449.223397 0.997080597284 0.517982843384 /home/vagrant/ctf/ctext_T_4.guess [255, 161, 182, 73, 254] [161, 47, 255, 162, 182]
5 176818.247874 0.991222985868 0.412260920422 /home/vagrant/ctf/ctext_T_5.guess [162, 73, 182, 255, 94, 47] [161, 47, 255, 162, 182]
6 174234.387473 0.987603807918 0.422144562838 /home/vagrant/ctf/ctext_T_6.guess [182, 162, 254, 255, 32, 47, 118] [161, 47, 255, 162, 182]
7 175761.662079 0.992223696543 0.424924719852 /home/vagrant/ctf/ctext_T_7.guess [0, 161, 162, 43, 164, 93, 182, 255] [161, 47, 

In [391]:
hex(200)

'0xc8'

In [392]:
0xc8

200

In [452]:
import os, sys, stat
import subprocess


import os
import stat



def make_bin(in_patch):
    in_binary = "./ctf/vigtor"
    #in_patch = #sys.argv[2] #"./ctf/vigtor.bin"

    out_binary = in_patch + ".patched"

    offset = 0x00000330 #int(sys.argv[3], 16) #0x00000330
    size =  0x000884f4 #int(sys.argv[4],16) #0x000884f4

    binary = open(in_binary, "rb").read()
    patch = open(in_patch, "rb").read() #G = ['A'] * 63066

    def write_to_text(main_file, filetowrite, offset,size):
        k = 0
        s = 0
        print len(filetowrite), size
        assert(len(filetowrite) == size)
        Y = []
        for b in main_file:
            k += 1
            if k >= offset + 1 and k <= offset + size:
                t = filetowrite[s]
                s += 1
                Y.append(t)
            else:
                Y.append(b)
        return Y

    Y = write_to_text(binary, patch, offset, size)
    newFile = open(out_binary, "wb")
    print out_binary
    r = bytearray(Y)
    newFile.write(r)
    newFile.close()
    st = os.stat(out_binary)
    os.chmod(out_binary, st.st_mode | stat.S_IEXEC)
    output = subprocess.Popen([out_binary])
    output.wait()
    print output.returncode